# Çoklu Doğrusal Regresyon + Flask GUI (Ödev)

> **Adınız:**  
> **Soyadınız:**  
> **Okul Numaranız:**  
> **GitHub Repo Bağlantısı:** https://github.com/kullanici_adi/MLP_Flask

Bu notebook, aşağıdaki adımları **net şekilde** göstermek için hazırlanmıştır:
1. Veri ön işleme (feature seçimi, missing values, kategorik encoding, opsiyonel scaling)
2. Backward Elimination (p-value ile)
3. Model kurulumu + değerlendirme (R², MAE, MSE)
4. Modeli `.pkl` kaydetme (Flask arayüzünde kullanmak için)


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import statsmodels.api as sm
import pickle

pd.set_option("display.max_columns", 200)


## 1) Veri setini yükle

Bu şablonda örnek olarak `data/insurance.csv` kullanılıyor (Medical Cost Personal Dataset).

- Hedef (target): `charges`
- Özellikler (<=10): `age, sex, bmi, children, smoker, region`

> Kendi veri setinle devam edeceksen, **hedef** ve **özellik** isimlerini burada güncelle.


In [ ]:
DATA_PATH = "data/insurance.csv"
TARGET_COL = "charges"
RAW_FEATURES = ["age", "sex", "bmi", "children", "smoker", "region"]  # <=10
CATEGORICAL_COLS = ["sex", "smoker", "region"]
NUMERIC_COLS = ["age", "bmi", "children"]

df = pd.read_csv(DATA_PATH)
df.head()


In [ ]:
df.info()

## 2) Feature seçimi

Ödev şartı: **maksimum 10 öznitelik**.
- Gereksiz/tekrarlı sütunları çıkar.
- Hedef sütunu (y) ayrı tutulur.


In [ ]:
df = df[RAW_FEATURES + [TARGET_COL]].copy()
df.head()


## 3) Missing values analizi ve işlem

- Eksik değerleri kontrol et.
- Silme veya doldurma (mean/median/mode) yöntemlerinden birini seç.
- **Neden** o yöntemi seçtiğini yaz.


In [ ]:
df.isna().sum()

In [ ]:
def fill_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if df[c].isna().sum() == 0:
            continue
        if pd.api.types.is_numeric_dtype(df[c]):
            df[c] = df[c].fillna(df[c].median())
        else:
            df[c] = df[c].fillna(df[c].mode(dropna=True)[0])
    return df

df = fill_missing_values(df)
df.isna().sum()


## 4) Kategorik veriler: Encoding

Burada `pandas.get_dummies(drop_first=True)` kullanıyoruz (One-Hot).
- `drop_first=True` dummy trap riskini azaltır.
- Alternatif: Label Encoding (sıralı kategori yoksa genelde One-Hot daha uygundur).


In [ ]:
X_raw = df[RAW_FEATURES]
y = df[TARGET_COL].astype(float)

X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.2, random_state=42
)

X_train = pd.get_dummies(X_train_raw, columns=CATEGORICAL_COLS, drop_first=True)
X_test = pd.get_dummies(X_test_raw, columns=CATEGORICAL_COLS, drop_first=True)

# test kolonlarını train'e hizala
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

X_train.head()


## 5) Backward Elimination (p-value)

Adımlar:
1. Tüm özelliklerle OLS modeli kur.
2. p-value > 0.05 olan en büyük p-value'lu sütunu çıkar.
3. Tekrar et.


In [ ]:
def backward_elimination(X: pd.DataFrame, y: pd.Series, sl: float = 0.05):
    cols = list(X.columns)
    X_ = sm.add_constant(X, has_constant="add")

    while True:
        model_ols = sm.OLS(y, X_).fit()
        pvals = model_ols.pvalues.drop("const", errors="ignore")
        max_p = pvals.max()

        if max_p <= sl:
            break

        worst_feature = pvals.idxmax()
        cols.remove(worst_feature)
        X_ = sm.add_constant(X[cols], has_constant="add")

    final_model = sm.OLS(y, X_).fit()
    return cols, final_model

selected_cols, ols_model = backward_elimination(X_train, y_train, sl=0.05)
selected_cols


In [ ]:
ols_model.summary()

## 6) Model eğitimi ve değerlendirme

Metrikler:
- **R²**
- **MAE**
- **MSE**


In [ ]:
lr = LinearRegression()
lr.fit(X_train[selected_cols], y_train)

y_pred = lr.predict(X_test[selected_cols])

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

r2, mae, mse


## 7) Modeli .pkl olarak kaydetme

Flask tarafında aynı dönüşümleri yapabilmek için:
- ham feature isimleri
- kategorik sütunlar
- numeric sütunlar
- seçilen (encode+elimination sonrası) kolonlar
da kaydedilir.


In [ ]:
bundle = {
    "model": lr,
    "raw_features": RAW_FEATURES,
    "categorical_cols": CATEGORICAL_COLS,
    "numeric_cols": NUMERIC_COLS,
    "model_columns": selected_cols,
}
with open("model.pkl", "wb") as f:
    pickle.dump(bundle, f)

print("model.pkl kaydedildi ✅")


## 8) Flask Arayüz

Repo içinde:
- `app.py` : Flask uygulaması
- `templates/index.html` : form ve sonuç gösterimi

Çalıştırma:
```bash
python model.py   # modeli eğitip model.pkl üret
python app.py     # Flask
```
